# Race/ethnicity by year

## Setup

### Packages

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc
import matplotlib.pyplot as plt
import time
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Pulling Existing Catalog from AGO

In [2]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
csv_item = gis.content.get(catalog_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"CSV file downloaded successfully: {catalog_path}")

CSV file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


### Paths

In [3]:
INPUT_DATA_PATH = "inputData/censusPep_county_char.csv"

TEMP_OUTPUT_CSV_PATH ="outputData/TEMP.csv"

REGIONAL_SUMMARY_DATA_PATH = "outputData/censusPep_county_char_15Region.csv"

REGIONAL_SUMMARY_DATA_LABELED_PATH = "outputData/censusPep_county_char_labeled_15Region.csv"

data1 = pd.read_csv(INPUT_DATA_PATH)

data1.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

data1.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
0,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2000,1
1,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2001,1
2,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2002,1
3,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2003,1
4,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2004,1


## Filtering

### Filtering for 15 county

In [4]:
# List of counties to filter by
selected_counties = [
    "Delaware", "Fairfield", "Fayette", "Franklin", "Hocking", "Knox", "Licking", 
    "Logan", "Madison", "Marion", "Morrow", "Perry", "Pickaway", "Ross", "Union"
]

selected_years = [
    2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
]

# Filter the dataset for the selected counties
data2 = data1[data1['COUNTY'].isin(selected_counties)]

# Filter the dataset for the selected counties
data2 = data2[data2['YEAR'].isin(selected_years)]

# Filter out rows where 'SOURCE' is 'INT' and 'YEAR' is '2010'
data2 = data2[~((data2['SOURCE'] == 'INT') & (data2['YEAR'] == '2010'))]

data2.to_csv(TEMP_OUTPUT_CSV_PATH, index=False)

# Display the first few rows of the filtered data
data2.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
178210,39041,Ohio,Delaware,F,AA,0 to 4,INT,2012,2010,389
178221,39041,Ohio,Delaware,F,AA,10 to 14,INT,2012,2010,274
178232,39041,Ohio,Delaware,F,AA,15 to 19,INT,2012,2010,187
178243,39041,Ohio,Delaware,F,AA,20 to 24,INT,2012,2010,139
178254,39041,Ohio,Delaware,F,AA,25 to 29,INT,2012,2010,278


### Summing by 15 County

In [5]:
# Group by the relevant columns and sum the 'POP' variable
data2_1 = data2.groupby(
    ['SEX', 'RACE', 'AGEGROUP', 'YEAR']
)['POP'].sum().reset_index()

# Add a new column for the COUNTY identifier
data2_1['COUNTY'] = '15County'

# Rearrange columns to match the original dataset order
data2_1 = data2_1[['COUNTY', 'SEX', 'RACE', 'AGEGROUP','YEAR', 'POP']]

# Append the new '15County' entries to the filtered dataset
data3 = pd.concat([data2, data2_1], ignore_index=True)

data3.to_csv(REGIONAL_SUMMARY_DATA_PATH, index=False)

# Display the first few rows of the updated data
data3.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
0,39041.0,Ohio,Delaware,F,AA,0 to 4,INT,2012.0,2010,389
1,39041.0,Ohio,Delaware,F,AA,10 to 14,INT,2012.0,2010,274
2,39041.0,Ohio,Delaware,F,AA,15 to 19,INT,2012.0,2010,187
3,39041.0,Ohio,Delaware,F,AA,20 to 24,INT,2012.0,2010,139
4,39041.0,Ohio,Delaware,F,AA,25 to 29,INT,2012.0,2010,278


### Filtering for sex totals and summing for different races

In [6]:
# Filter the dataset where SEX == 'TOT'
data3_1 = data3[data3['SEX'] != 'TOT']

# Group by the relevant columns and sum the 'POP' variable
data4 = data3_1.groupby(
    ['COUNTY', 'RACE', 'YEAR']
)['POP'].sum().reset_index()

data4.to_csv(REGIONAL_SUMMARY_DATA_PATH, index=False)

# Display the first few rows of the updated data
data4.head()

,COUNTY,RACE,YEAR,POP
0,15County,AA,2010,121054
1,15County,AA,2011,63769
2,15County,AA,2012,66927
3,15County,AA,2013,71774
4,15County,AA,2014,77219


### Combining RACE NA

In [7]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
hna_nhna_data = data4[data4['RACE'].isin(['HNA', 'NHNA'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = hna_nhna_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'NA'


# Step 4: Append the new rows to the original data
data5 = pd.concat([data4, combined_pop], ignore_index=True)

data5.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

### Combining hispanic 

In [8]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
h_data = data5[data5['RACE'].isin(['HNA', 'HBA', 'HAA', 'HIA', 'HWA', 'HTOM'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = h_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'HH'

# Step 4: Append the new rows to the original data
data6 = pd.concat([data5, combined_pop], ignore_index=True)

data6.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

### Combining other races (NHIA + NHNA)

In [9]:
# Step 1: Filter data for RACE == 'HNA' and RACE == 'NHNA'
other_data = data6[data6['RACE'].isin(['NHIA', 'NHNA'])]

# Step 2: Group by COUNTY, SOURCE, VINTAGE, and YEAR, and sum the POP
combined_pop = other_data.groupby(['COUNTY', 'YEAR'], as_index=False)['POP'].sum()

# Step 3: Add a new column for RACE with value 'NA'
combined_pop['RACE'] = 'OTHER'

# Step 4: Append the new rows to the original data
data7 = pd.concat([data6, combined_pop], ignore_index=True)

data7.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

# Display the first few rows of the updated dataframe
data7.head()

,COUNTY,RACE,YEAR,POP
0,15County,AA,2010,121054
1,15County,AA,2011,63769
2,15County,AA,2012,66927
3,15County,AA,2013,71774
4,15County,AA,2014,77219


### Proportions

In [10]:
# Filter the dataset to include only the specified RACE categories, including 'TOT'
race_categories = {'NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER', 'TOT'}
filtered_data = data7[data7['RACE'].isin(race_categories)]

# Pivot data to have RACE as columns for easier calculation of proportions
pivot_df = filtered_data.pivot_table(index=['COUNTY', 'YEAR'], columns='RACE', values='POP', fill_value=0)

# Calculate proportions by dividing each race population by the total population (TOT)
for race in ['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER']:
    pivot_df[f'{race}_proportion'] = pivot_df[race] / pivot_df['TOT']

# Reset index for a more conventional DataFrame format
proportions_df = pivot_df.reset_index()

# Display the resulting DataFrame
proportions_df[['COUNTY', 'YEAR'] + [f'{race}_proportion' for race in ['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER']]]


df=proportions_df


pop_df = df.melt(id_vars=['COUNTY', 'YEAR'], value_vars=['HH', 'NHAA', 'NHBA', 'NHTOM', 'NHWA', 'OTHER'], 
                 var_name='RACE', value_name='POP')

# Melt the proportion columns to get 'POP_PROP' values for each RACE
prop_df = df.melt(id_vars=['COUNTY', 'YEAR'], 
                  value_vars=['HH_proportion', 'NHAA_proportion', 'NHBA_proportion', 'NHTOM_proportion', 'NHWA_proportion', 'OTHER_proportion'], 
                  var_name='RACE_PROP', value_name='POP_PROP')

# Adjust RACE column in the proportion data to match the format in pop_df
prop_df['RACE'] = prop_df['RACE_PROP'].str.replace('_proportion', '')

# Merge the two DataFrames on COUNTY, YEAR, and RACE to get POP and POP_PROP in a single DataFrame
data8 = pd.merge(pop_df, prop_df[['COUNTY', 'YEAR', 'RACE', 'POP_PROP']], on=['COUNTY', 'YEAR', 'RACE'])

data8.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

data8.head()

,COUNTY,YEAR,RACE,POP,POP_PROP
0,15County,2010,HH,142849.0,0.032664
1,15County,2011,HH,73501.0,0.033310
2,15County,2012,HH,75822.0,0.034059
3,15County,2013,HH,78700.0,0.034933
4,15County,2014,HH,81701.0,0.035826


### Label map

In [11]:
# Now, filter out records where RACE is NOT in the specified set
filtered_data = data8[data8['RACE'].isin(['NHAA', 'NHBA', 'NHWA', 'NHTOM', 'HH', 'OTHER'])]

# Define a comprehensive mapping of race abbreviations to full race names
race_mapping = {
    "NHAA": "Asian",
    "HH": "Hispanic or Latio (any race)",
    "NHBA": "Black or African American",
    "OTHER": "Some other race alone",
    "NHTOM": "Two or more races",
    "NHWA": "White"
}

# Apply the mapping to the 'RACE' column
filtered_data['RACE'] = filtered_data['RACE'].map(race_mapping).fillna(filtered_data['RACE'])

filtered_data.to_csv(REGIONAL_SUMMARY_DATA_LABELED_PATH, index=False)

# Display the first few rows of the updated data
filtered_data.head()

,COUNTY,YEAR,RACE,POP,POP_PROP
0,15County,2010,Hispanic or Latio (any race),142849.0,0.032664
1,15County,2011,Hispanic or Latio (any race),73501.0,0.033310
2,15County,2012,Hispanic or Latio (any race),75822.0,0.034059
3,15County,2013,Hispanic or Latio (any race),78700.0,0.034933
4,15County,2014,Hispanic or Latio (any race),81701.0,0.035826


## Garphing

### Creating images for each region

In [12]:
import matplotlib.pyplot as plt

# Load the uploaded file to review its content
file_path =REGIONAL_SUMMARY_DATA_LABELED_PATH

# Load the data from the CSV file
data = pd.read_csv(file_path)

# Set up output directory
output_dir = 'outputData/plots'

os.makedirs(output_dir, exist_ok=True)

# Dictionary to store COUNTY to itemID mapping
county_item_mapping = {}

# Iterate over each county
for county in data['COUNTY'].unique():
    county_data = data[data['COUNTY'] == county]
    
    # Ensure 'YEAR', 'POP_PROP', and 'RACE' columns are present
    county_data = county_data[['YEAR', 'POP_PROP', 'RACE']]
    
    # Aggregate to handle duplicates by taking the mean
    county_data = county_data.groupby(['YEAR', 'RACE'], as_index=False).agg({'POP_PROP': 'mean'})
    
    # Pivot data to have 'RACE' as columns for plotting
    county_data_pivot = county_data.pivot(index='YEAR', columns='RACE', values='POP_PROP')
    
    # Plot each county's data with adjacent bars
    plt.figure(figsize=(16, 9))
    bar_width = 0.15  # Reduced width for better spacing
    x_positions = range(len(county_data_pivot.index))
    
    # Improved color palette
    colors = ['#365072', '#34617A', '#0075BF', '#00B2BF', '#2A7F67', '#5AB65F']
    
    for i, (race, color) in enumerate(zip(county_data_pivot.columns, colors)):
        plt.bar(
            [x + i * bar_width for x in x_positions],
            county_data_pivot[race],
            width=bar_width,
            label=race,
            color=color
        )
    
    # Adjust text sizes
    plt.title(f"{county} Population Over Time (By Race/Ethnicity)", fontsize=25, pad=20)  # Larger title with padding
    plt.ylabel("Population Proportion", fontsize=20)  
    plt.xticks(
        [x + bar_width * (len(county_data_pivot.columns) - 1) / 2 for x in x_positions], 
        county_data_pivot.index, 
        rotation=30,  # Slightly tilt labels
        fontsize=18  # Larger tick labels
    )
    plt.yticks(fontsize=18)  # Larger y-axis tick labels
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add light grid lines for y-axis
    plt.legend(
        loc='upper center', 
        bbox_to_anchor=(0.5, -0.12),  # Add more padding below the chart for legend
        ncol=3,  # Limit to 3 columns for compactness
        fontsize=20,  # Larger legend text
        title_fontsize=16  # Larger legend title
    )
    
    # Save the plot as a PNG file and add to mapping
    plot_path = os.path.join(output_dir, f"{county}_population_over_time_adjacent_V2.png")
    plt.savefig(plot_path, bbox_inches='tight')  # Use bbox_inches to avoid cropping
    plt.close()
    
    # Placeholder ID for demonstration as upload to AGO isn't performed here
    county_item_mapping[county] = f"simulated_item_id_for_{county}"


# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_df['TilesetID'] = 14
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)

mapping_file_path

'AGO_County_ItemID_Mapping.csv'

### Uploading images to AGO

In [13]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Directory where images are saved
output_dir = 'outputData/plots'

# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Population proportion over time by race for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_df['TilesetID'] = 14
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)



Deleted existing item with title: 15County population over time adjacent V2


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15County_population_over_time_adjacent_V2.png with item ID: 495a48a08093402da80e008a31bfc032 and set it to public
Deleted existing item with title: Delaware population over time adjacent V2


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_population_over_time_adjacent_V2.png with item ID: 813795f6eb2946189708ac135b670891 and set it to public
Deleted existing item with title: Fairfield population over time adjacent V2


...

Uploaded Pickaway_population_over_time_adjacent_V2.png with item ID: 13c14eb0f0724ad3bed830d1debfcff0 and set it to public
Deleted existing item with title: Ross population over time adjacent V2


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_population_over_time_adjacent_V2.png with item ID: b90024f1a97e4e44bec6dfd277d16f65 and set it to public
Deleted existing item with title: Union population over time adjacent V2


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_population_over_time_adjacent_V2.png with item ID: c505d73cc4d04246817020e2c62560bf and set it to public


### Update Metadata Local

In [14]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path)

# Ensure consistent naming
mapping_df['COUNTY'] = mapping_df['COUNTY'].replace('15-County', '15County')

# Select only necessary columns from mapping_df
selected_mapping_df = mapping_df[['COUNTY', 'TilesetID', 'itemID']]

# Merge while keeping all columns from catalog_df
merged_df = catalog_df.merge(
    selected_mapping_df,
    left_on=['GeographyName', 'TilesetID'],
    right_on=['COUNTY', 'TilesetID'],
    how='left'
)

# Update 'ThumbnailURL' using 'itemID' where applicable
merged_df['ThumbnailURL'] = merged_df['itemID'].combine_first(merged_df['ThumbnailURL'])

# Drop unnecessary columns from the merged dataset
merged_df = merged_df.drop(columns=['COUNTY', 'itemID'], errors='ignore')

# Ensure the final DataFrame retains only original catalog_df columns (plus the updated ThumbnailURL)
expected_columns = list(catalog_df.columns)  # Retain all original columns
if 'ThumbnailURL' not in expected_columns:
    expected_columns.append('ThumbnailURL')  # Ensure ThumbnailURL is included

merged_df = merged_df[expected_columns]  # Reorder columns to match the original catalog

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
merged_df.to_excel(updated_catalog_path, index=False)

print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


### Pushing changes to AGO

In [15]:
from arcgis.gis import GIS

# Title or Item ID of the existing XLSX file in AGO
item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
local_xlsx_path = updated_catalog_path

# --- Search for the Existing Item ---
items = gis.content.search(item_id, item_type="Microsoft Excel")
if not items:
    print("Error: No matching XLSX file found.")
else:
    item = items[0]  # Assuming the first match is correct
    print(f"Found item: {item.title} ({item.id})")

    # --- Update the Item with the New File ---
    item.update({}, local_xlsx_path)

    print(f"Successfully replaced {item.title} with the new file.")


Found item: morpc_insights_catalog (c396b9d0db724d1f870fa47f1d1dc1ce)
Successfully replaced morpc_insights_catalog with the new file.
